In [18]:
# Create connection
import mysql.connector

connection = mysql.connector.connect(
    user='root',
    password='password',
    host='localhost',
    database='db'
)

# Make sure connection established
if connection.is_connected():
    print("Connected")
else:
    print("Failed to connect")

Connected


## Queries

#### Roll-Up/Drill Down

Roll up car sales by car make and model.

In [19]:
cursor = connection.cursor()

In [20]:
query = 'SELECT c.company, c.model, SUM(cs.price) AS TotalSales FROM car c JOIN car_sales cs ON c.id = cs.car GROUP BY c.company, c.model;'
cursor.execute(query)
cursor.fetchall()

[('ford', 'expedition', Decimal('4838250')),
 ('dodge', 'durango', Decimal('5878455')),
 ('cadillac', 'eldorado', Decimal('9725407')),
 ('toyota', 'celica', Decimal('4584448')),
 ('acura', 'tl', Decimal('6139019')),
 ('mitsubishi', 'diamante', Decimal('9308151')),
 ('toyota', 'corolla', Decimal('2878632')),
 ('mitsubishi', 'galant', Decimal('3255171')),
 ('chevrolet', 'malibu', Decimal('8885102')),
 ('ford', 'escort', Decimal('980483')),
 ('acura', 'rl', Decimal('8778638')),
 ('nissan', 'pathfinder', Decimal('8792523')),
 ('subaru', 'forester', Decimal('5738743')),
 ('hyundai', 'accent', Decimal('634518')),
 ('toyota', 'landcruiser', Decimal('2349022')),
 ('honda', 'accord', Decimal('6597822')),
 ('toyota', '4runner', Decimal('1695515')),
 ('infiniti', 'i30', Decimal('5717040')),
 ('audi', 'a4', Decimal('2009977')),
 ('volkswagen', 'jetta', Decimal('12287736')),
 ('dodge', 'viper', Decimal('7417262')),
 ('buick', 'regal', Decimal('4760614')),
 ('chrysler', 'lhs', Decimal('9097717')),
 

Roll up car sales by year and fiscal quarter.

In [21]:
query = 'CREATE VIEW salesbyfuelincome AS SELECT d.year, d.fiscal_quarter, SUM(cs.price) AS total_price FROM car_sales cs JOIN date d ON cs.date = d.id GROUP BY d.year, d.fiscal_quarter;'
cursor.execute(query)
cursor.fetchall()

[]

#### Slice

Select sales where the CPI value is greater than 120.

In [22]:
query = 'CREATE VIEW salescpi120 AS SELECT cs.*FROM car_sales cs JOIN date d ON cs.date = d.id JOIN cpi c ON d.id = c.date WHERE c.cpi_value > 120;'
cursor.execute(query)
cursor.fetchall()

ProgrammingError: 1050 (42S01): Table 'salescpi120' already exists

#### Dice

Select sales where the city miles per gallon of the car purchased is less than 19 and the income of the customer is less than 60000

In [34]:
cursor.fetchall()
query = '''CREATE VIEW SalesbyMpgIncome AS
SELECT cs.*
FROM car_sales cs
JOIN car c ON cs.car = c.id
JOIN customer cu ON cs.customer = cu.id
WHERE c.city_mpg < 19 AND cu.annual_income < 60000;
'''
cursor.execute(query)

Select sales where the engine fuel type is "regualar unleaded" and customer annual income is > 82000

In [28]:
query = '''
CREATE VIEW salesbyfuelincome AS
SELECT cs.*
FROM car_sales cs
JOIN car c ON cs.car = c.id
JOIN customer cu ON cs.customer = cu.id
WHERE c.engine_fuel_type = 'regular unleaded' AND cu.annual_income > 82000;
'''
cursor.execute(query)

## Combining OLAP Operations

### Slice + Dice + Roll Up

In [ ]:
# Within the past 3 months for Toyota models grouped by month and year
query = "SELECT d.month, d.year, c.company, c.model, SUM(cs.price) AS TotalSales FROM car c JOIN car_sales cs ON c.id = cs.car JOIN date d ON cs.date = d.id WHERE (d.year = 2023 AND d.month >= 10 AND d.month <= 12 AND d.day <= 31) AND c.company = 'Toyota' GROUP BY d.year, d.month, c.model;"

# Within the past year for Toyota models grouped by month and year
query = "SELECT d.month, d.year, c.company, c.model, SUM(cs.price) AS TotalSales FROM car c JOIN car_sales cs ON c.id = cs.car JOIN date d ON cs.date = d.id WHERE d.year = 2023 AND c.company = 'Toyota' GROUP BY d.year, d.month, c.model;"


### Slice + Dice

In [ ]:
# Female
query = "SELECT cst.annual_income, cr.engine_fuel_type FROM customer cst JOIN car_sales cs ON cst.id = cs.customer JOIN car cr ON cs.car = cr.id WHERE cst.gender = 'Female';"

# Male
query = "SELECT cst.annual_income, cr.engine_fuel_type FROM customer cst JOIN car_sales cs ON cst.id = cs.customer JOIN car cr ON cs.car = cr.id WHERE cst.gender = 'Male';"

### Dice + Roll Up

In [ ]:
# Get number of sales for a month for all Toyotas
query = "SELECT d.month, c.company, COUNT(cs.id), AVG(cs.price) AS TotalSales FROM car c JOIN car_sales cs ON c.id = cs.car JOIN date d ON cs.date = d.id WHERE c.company = 'Toyota' GROUP BY d.month, c.company;"

### Drill Down + Slice

In [ ]:
query = "SELECT d.month, d.day, c.company, COUNT(cs.id), AVG(cs.price) AS TotalSales FROM car c JOIN car_sales cs ON c.id = cs.car JOIN date d ON cs.date = d.id WHERE c.company = 'Toyota' GROUP BY d.month, d.day, c.company;"